In [1]:
import requests
import pandas as pd
import os
import json

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from pyspark.sql.functions import current_date, current_timestamp

from pyspark.sql import SparkSession

In [2]:
os.getcwd()

'/home/jovyan/work'

In [3]:
input_url = "https://api.openbrewerydb.org/v1/breweries"
prefix = "/home/jovyan/work/layers/bronze"
file_name = "bronze_breweries.json"
output_path = os.path.join(prefix, file_name)

In [4]:
output_path

'/home/jovyan/work/layers/bronze/bronze_breweries.json'

In [5]:
retry_strategy = Retry(
    total=2,
    backoff_factor=1,
    status_forcelist=[400, 401, 402, 403, 429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retry_strategy)

session = requests.Session()
session.mount("https://", adapter)
session.mount("http://", adapter)

try:
    response = session.get(input_url)
    response.raise_for_status()
    if response.status_code == 200:
        dados_json = response.json()
    else:
        error_message = f"Error accessing data: {response.status_code}"
        raise Exception(error_message)
except Exception as e:
    error_message = f"Error processing JSON: {e}"
    raise Exception(error_message)

conteudo_json = json.dumps(dados_json, indent=4, ensure_ascii=False)

# Save raw JSON to bronze layer
#try:
#    with open(output_path, "w", encoding="utf-8") as f:
#        f.write(conteudo_json)
#except Exception as e:
#    print(f"Error saving file: {e}")

In [6]:
df_pandas = pd.json_normalize(dados_json)

In [7]:
df_pandas.head()

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,None,Norman,Oklahoma,73069-8224,United States,-97.468182,35.257389,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,None,None,Austin,Texas,78745-1197,United States,NaN,NaN,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,micro,8100 Washington Ave,None,None,Mount Pleasant,Wisconsin,53406-3920,United States,-87.883364,42.720108,2624847553,https://www.1ofusbrewing.com,Wisconsin,8100 Washington Ave
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,None,None,Bend,Oregon,97701-9847,United States,-121.281706,44.086835,5415851007,http://www.10barrel.com,Oregon,62970 18th St
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,None,None,Bend,Oregon,97703-2465,United States,-121.328802,44.057565,5415851007,None,Oregon,1135 NW Galveston Ave Ste B
